In [2]:
from pysvn_tuto import get_svn_logs
path = r"D:\dev\AutoPlanning\trunk\AP_trunk_pure"
logs = get_svn_logs(path)

3902 개의 로그


In [14]:
import numpy as np

In [15]:
from collections import defaultdict
file_path_map = defaultdict(list)

for log in logs:
    changed_path_info = log['changed_paths']
    if changed_path_info:
        for changed_path_info in log['changed_paths']:
            # print(changed_path_info)
            file_path_map[changed_path_info['path']].append(log)
    else:
        print(log)

        

{'revision': '5901', 'author': 'ldongoh', 'date': datetime.datetime(2022, 8, 30, 1, 42, 17, 502696), 'message': '[FIXED]\n- 외부 external lib 링크 변경', 'changed_paths': None}
{'revision': '5900', 'author': 'ldongoh', 'date': datetime.datetime(2022, 8, 30, 1, 34, 45, 999714), 'message': '[FIXED]\n- 외부 external library 링크 수정', 'changed_paths': []}
{'revision': '5899', 'author': 'ldongoh', 'date': datetime.datetime(2022, 8, 30, 1, 26, 27, 510046), 'message': '[FIXED]\n- 외부 external lib 링크 경로 수정', 'changed_paths': []}
{'revision': '5898', 'author': 'ldongoh', 'date': datetime.datetime(2022, 8, 30, 1, 17, 53, 627475), 'message': '[FIXED]\n- external library 링크 변경', 'changed_paths': []}
{'revision': '5879', 'author': 'ldongoh', 'date': datetime.datetime(2022, 8, 10, 23, 49, 49, 154015), 'message': '[CHANGED]\n- 번호 : X\n- 내용 : 불필요 tensorflow 외부라이브러리 링크 해제', 'changed_paths': []}
{'revision': '2701', 'author': 'ldongoh', 'date': datetime.datetime(2019, 12, 10, 0, 36, 6, 525621), 'message': '[BRANCH

In [16]:
# for path in file_path_map:
#     print(f'{path} chagne {len(file_path_map[path])}' )

In [17]:
path = r'D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APSurgicalGuide\ToolBarSurgicalGuide.h'
rv_list = file_path_map[path]

In [18]:
path_base = [ data['revision'] for data in rv_list]
log_base = [ data['revision'] for data in rv_list]
msg_list = [ data['message'] for data in rv_list]


In [19]:
"""
1. 먼저 로그에서 메시지와 해당 리비전에 변경된 파일을 리스트로 
MSG
chageFiles = [fileName1,  fileName2, fileName3 .... ]

2. 파일은 유니크 하므로 파일들을 enum화 한다.
chageFiles = [a, b, c, d ...]

3. 그러면 chageFiles은 일종의 클러스트링이나 좌표적인 의미를 가질지도??

4. 따라서 메시지를 벡터화와 파일을 연결하면??
"""

'\n1. 먼저 로그에서 메시지와 해당 리비전에 변경된 파일을 리스트로 \nMSG\nchageFiles = [fileName1,  fileName2, fileName3 .... ]\n\n2. 파일은 유니크 하므로 파일들을 enum화 한다.\nchageFiles = [a, b, c, d ...]\n\n3. 그러면 chageFiles은 일종의 클러스트링이나 좌표적인 의미를 가질지도??\n\n4. 따라서 메시지를 벡터화와 파일을 연결하면??\n'

In [20]:
# 1. 메시지, 파일 연결
msg_files = []
for log in logs:
    changed_path_infos = log['changed_paths']
    if changed_path_infos:
        path_list = [path_info['path'] for path_info in changed_path_infos]
        msg_files.append( (log['message'], path_list) )
    else:
        print(log)

print(msg_files[22])

{'revision': '5901', 'author': 'ldongoh', 'date': datetime.datetime(2022, 8, 30, 1, 42, 17, 502696), 'message': '[FIXED]\n- 외부 external lib 링크 변경', 'changed_paths': None}
{'revision': '5900', 'author': 'ldongoh', 'date': datetime.datetime(2022, 8, 30, 1, 34, 45, 999714), 'message': '[FIXED]\n- 외부 external library 링크 수정', 'changed_paths': []}
{'revision': '5899', 'author': 'ldongoh', 'date': datetime.datetime(2022, 8, 30, 1, 26, 27, 510046), 'message': '[FIXED]\n- 외부 external lib 링크 경로 수정', 'changed_paths': []}
{'revision': '5898', 'author': 'ldongoh', 'date': datetime.datetime(2022, 8, 30, 1, 17, 53, 627475), 'message': '[FIXED]\n- external library 링크 변경', 'changed_paths': []}
{'revision': '5879', 'author': 'ldongoh', 'date': datetime.datetime(2022, 8, 10, 23, 49, 49, 154015), 'message': '[CHANGED]\n- 번호 : X\n- 내용 : 불필요 tensorflow 외부라이브러리 링크 해제', 'changed_paths': []}
{'revision': '2701', 'author': 'ldongoh', 'date': datetime.datetime(2019, 12, 10, 0, 36, 6, 525621), 'message': '[BRANCH

In [21]:
# 2. 파일 명을 enum 으로
file_to_enum = {file: idx for idx, file in enumerate(file_path_map)}

# print("Unique Files:", unique_files)
# for path in file_to_enum:
#     print(f"{file_to_enum[path]} :{path}")


In [22]:
#3.1 파일 경로 리스트를 정규화
def normalize_path(str_path_list:[str]):
    n_data = [0 for _ in range(len(file_to_enum))]
    for path in str_path_list:
        e_idx = file_to_enum[path]
        n_data[e_idx] = 1
    return n_data

len(file_to_enum)

138371

In [23]:
#3.2 메시지 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer

str_msg_list = [log['message'] for log in  logs]

vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(str_msg_list)

msg_vecotr_map = {}
for idx, str_msg in enumerate(str_msg_list):
    msg_vecotr_map[str_msg] = x[idx]
    



def tokenize(txt):
    '''
    to do:
    전처리 / 필터링 등 뭔가 해야할건 많음
    일단은 간단하게 태스트 (TF-IDF : 빈도수 기반)
    '''
    #<'scipy.sparse._csr.csr_matrix'> to <'numpy.ndarray'>
    if txt in msg_vecotr_map:
        return msg_vecotr_map[txt].toarray()
    else:
        vectorizer.



# for msg in msg_vecotr_map:
#     print(msg)
#     print(msg_vecotr_map[msg].shape)


[CHANGED] [Design] 2DCS CT 출력
(1, 6538)
Bug #4348 : [UI-UX] [Scan-alignment] Slider tooltip 미표출

수정원인 : 정합 모드에 따라 Tooltip 출력 처리로 인한 tooltip 미표출
해결방법 : tooltip은 CT 슬라이더 확장시 모드 상관 없이 표출
검증방법 : 툴팁 표출 확인
(1, 6538)
[fixed] toothremove scandata set 조건 추가
(1, 6538)
[CHANGED]
- 번호 : 4180, 4329
- 내용 : CrownDesign Copy 변경 UI 구성
(1, 6538)
[ADDED]
- 번호 : 4180, 4329
- 내용 : CrownDesign Copy 변경 이미지 리소스 추가
(1, 6538)
Bug #4328 [수술가이드] 브러쉬 부분 제거 버튼 선택 초기 상태와 불일치 함(누락 추가)

수정원인 : 브러쉬 제거 버튼 선택시,  초기 상태와 불일치
해결방법 : fill, cut, reset 버튼 활성화에 대한 조건 정리
검증방법 : 재현 절차와 동일하게 검증
(1, 6538)
Bug #4328 [수술가이드] 브러쉬 부분 제거 버튼 선택 초기 상태와 불일치 함

수정원인 : 브러쉬 제거 버튼 선택시,  초기 상태와 불일치
해결방법 : fill, cut, reset 버튼 활성화에 대한 조건 정리
검증방법 : 재현 절차와 동일하게 검증
(1, 6538)
Bug #4340 [덴탈아치,스캔정렬] 단면 Plate 출력 오류

수정원인 : plate 테두리가 가운데 고정으로 출력 됨
해결방법 : 이전 코드로 롤백
검증방법 : 정상적인 Plate 출력 확인
(1, 6538)
[FIXED]
- 내용 : Actuator CrownDeisgn DBL Click 함수 정의 추가
(1, 6538)
[fixed] tooth remove edit point 예외처리
(1, 6538)
[fixed]
- 번호 : 4297, 4342
- 내용 : 가이드맵 버그 수정
(1

In [36]:
vocab_list = list(vectorizer.vocabulary_.keys())
print(f"{len(vocab_list)} 개의 단어 벡터화")

6538 개의 단어 벡터화


In [24]:
import random as rand
# 3.3 메시지 백터화, 파일 경로 정규화 연결


rand.shuffle(msg_files)  #무작위 순서

n_msg_files = []

for pair in  msg_files:
    text_vector = tokenize(pair[0]) #(1, 6538)
    labels = normalize_path(pair[1]) #(1, 138371)
    n_pair = (text_vector, labels)
    n_msg_files.append( n_pair )

# print(text_vector.shape)
# print(labels)

In [25]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from tensorflow.keras.models import load_model

import sys

print("Python version")
print(sys.version)
print("Version info.")
print(sys.version_info)

# SavedModel 형식으로 저장된 모델 로드
loaded_model = load_model('first_train_models')

# 모델 요약 출력
loaded_model.summary()


Num GPUs Available:  1
Python version
3.7.1 (default, Oct 28 2018, 08:39:03) [MSC v.1912 64 bit (AMD64)]
Version info.
sys.version_info(major=3, minor=7, micro=1, releaselevel='final', serial=0)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 4096)              26783744  
                                                                 
 dense_5 (Dense)             (None, 1024)              4195328   
                                                                 
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 dense_7 (Dense)             (None, 138371)            70984323  
                                                                 
Total params: 102,488,195
Trainable params: 102,488,195
Non-trainable params: 0
___________________________________________

In [35]:
size = len(n_msg_files)
test_rate = 0.2
test_idx = int(size * test_rate)

x = [pair[0] for pair in n_msg_files]
y = [pair[1] for pair in n_msg_files]

x = np.vstack(x)
y = np.vstack(y)

print(x.shape)  # (n, 6538)
print(y.shape)  # (n, 138371)

train_x = x[:-test_idx]
train_y = y[:-test_idx]

test_x = x[-test_idx:]
test_y = y[-test_idx:]

predict_y=loaded_model.predict(test_x)

(3888, 6538)
(3888, 138371)


In [95]:
train_msg_files = msg_files[:-test_idx]
test_msg_files = msg_files[-test_idx:]

#대략적인 데이터의 예측 차이를 출력
def summary_predcit_info(index , n_x = None, n_y, org_set, predict_y):
    print(org_set[index][0])
     
    #n_x 입력시 단어 개수 정도? 
    if n_x:
        n_count = 0
        #소수점은 해당 단어가 호출되는 빈도 (전체 데이터셋 기준)
        for idx, n_data in enumerate(n_x[index]):
            if n_data:
                n_count += 1
        print(f"단어 수 {n_count}")
    
    print(f"수정 파일 수 {len(org_set[index][1])}")
            
    #예측 데이터
    n_count = 0
    for idx, n_data in enumerate(predict_y[index]):
        if 0.5 < n_data:
            n_count+=1
    
    print(f"예측 라벨링 개수 {n_count}")
    
    return n_count

#하나의 데이터를 상세하게 출력
def print_predcit_y(index , n_x, n_y, org_set, predict_y):
    print(org_set[index][0])
    print(f'->{n_x[index]} len {len(n_x[index])}')

    # 인덱스와 키를 뒤집음: 인덱스 -> 단어
    reversed_vocab = {i: word for word, i in vectorizer.vocabulary_.items()}
    
    #소수점은 해당 단어가 호출되는 빈도 (전체 데이터셋 기준)
    for idx, n_data in enumerate(n_x[index]):
        if n_data:
            print(f'{idx}:{n_data} -> ({reversed_vocab[idx]})' )
    
    print()
    for path in org_set[index][1]:
        print(f'{path} : {file_to_enum[path]}')
    
    print(f'->{y[index]} len {len(y[index])}')
    
    # 인덱스와 키를 뒤집음: 인덱스 -> 경로
    reversed_path = {i: p for p, i in file_to_enum.items()}
    
    for idx, n_data in enumerate(n_y[index]):
        if n_data:
            print(f'{idx}:{n_data} -> ({reversed_path[idx]})' ) 
            
            
    #예측 데이터
    print("=== predict ===")
    for idx, n_data in enumerate(predict_y[index]):
        if 0.5 < n_data:
            print(f'{idx}:{n_data} -> ({reversed_path[idx]})' ) 
            
# # for i in range(5):
# #     print_org_to_n(50 * i)
# #     print("========="*5)


is_label_predict_list = []
print(len(predict_y))
for idx, n_data in enumerate(predict_y):
    is_label = False
    for d in n_data:
        if 0.5 < d:
            is_label = True
            break
    if is_label:
        print_predcit_y(idx, test_x, test_y, test_msg_files, predict_y)
        is_label_predict_list.append(idx)
        
                
    


# predict_dict = defaultdict(list)
# for idx , d in enumerate(predict_y[5]):
#     predict_dict[d].append(idx)

# # for d in predict_dict:
# #     print(d)
    
    

777
Task #4111 [Surgical Guide] Bar/Window 사양 정의

-BarWindow 폴더 분리
-ToothRemove 수행시, Fixture 위치 이동시 Guide, Bar, Tag 데이터 모두 삭제
-UpdateSurface, 투명도 조절 분리(Guide, Bar, Tag)
-BarWindow 진입시 항상 Play 버튼
-Guide Creation, Bar and Window, ID Tag 진입시 선택된 LoopNum 가져옴
-Making Guide 수행 후 -> BarWindow, Tag 에 복사
->[0. 0. 0. ... 0. 0. 0.] len 6538
453:0.13505752572306753 -> (4111)
762:0.12912505380811334 -> (and)
837:0.33959131985831525 -> (bar)
838:0.38737516142434003 -> (barwindow)
1062:0.10975499637516016 -> (creation)
1358:0.06629599403304035 -> (fixture)
1443:0.30728646023547834 -> (guide)
1489:0.09411061115856392 -> (id)
1645:0.15121300139429128 -> (loopnum)
1664:0.15121300139429128 -> (making)
1950:0.14406408952417624 -> (play)
2302:0.06223364200286812 -> (surgical)
2323:0.3807180290912949 -> (tag)
2332:0.07330271653496769 -> (task)
2406:0.11568746829011436 -> (toothremove)
2470:0.15121300139429128 -> (updatesurface)
2554:0.18190748531962864 -> (window)
2647:0.15121300139429128 -> (가져옴)
3243:0.08

[ADDED] DoitInterface 생성

Dio3DAlgorithm 관련 Interface 이 곳에 탑재 예정
->[0. 0. 0. ... 0. 0. 0.] len 6538
738:0.22618032700150684 -> (added)
1222:0.45114964129778634 -> (dio3dalgorithm)
1252:0.41468739674154137 -> (doitinterface)
1549:0.26883841851656165 -> (interface)
2805:0.429820595540259 -> (곳에)
2832:0.2394004459048092 -> (관련)
4255:0.23533460244531243 -> (생성)
4844:0.3104789947600996 -> (예정)
6048:0.3274581338160166 -> (탑재)

D:\dev\AutoPlanning\trunk\AP_trunk_pure\DIOAutoPlanning.sln : 311
D:\dev\AutoPlanning\trunk\AP_trunk_pure\DoitInterface\DoitInterface.vcxproj : 484
D:\dev\AutoPlanning\trunk\AP_trunk_pure\DoitInterface\DoitInterface.vcxproj.filters : 576
D:\dev\AutoPlanning\trunk\AP_trunk_pure\DoitInterface\MeshUtil.cpp : 604
D:\dev\AutoPlanning\trunk\AP_trunk_pure\DoitInterface\MeshUtil.h : 605
D:\dev\AutoPlanning\trunk\AP_trunk_pure\DoitInterface\ReactPhysics3D Release.props : 50894
D:\dev\AutoPlanning\trunk\AP_trunk_pure\DoitInterface\stdafx.cpp : 85892
D:\dev\AutoPlanning\trunk\AP_

Task #3957 (AP) 2-4 : ID Tag 

-버튼 인터페이스(텍스트, 사이즈 변경)
->[0. 0. 0. ... 0. 0. 0.] len 6538
327:0.372158236051433 -> (3957)
770:0.3424523825445676 -> (ap)
1489:0.3166294332780763 -> (id)
2323:0.3202256693105996 -> (tag)
2332:0.2466225360613678 -> (task)
3837:0.23830083017383835 -> (버튼)
3872:0.17957727579920202 -> (변경)
4147:0.39331609645301296 -> (사이즈)
5134:0.29219090454259944 -> (인터페이스)
6068:0.3892234581322348 -> (텍스트)

D:\dev\AutoPlanning\trunk\AP_trunk_pure\AppFrame\UpdateMessages.h : 29
D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APSurgicalGuide\ActuatorSurgicalGuide.cpp : 47
D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APSurgicalGuide\CommandIDTag.cpp : 27
D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APSurgicalGuide\CommandIDTag.h : 207
D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APSurgicalGuide\ToolBarSurgicalGuide.cpp : 18
->[0 0 0 ... 0 0 0] len 138371
18:1 -> (D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APSurgicalGuide\ToolBarSurgicalGuide.cpp)
27:1 -> (D:\dev\AutoPlanning\trunk\AP_

In [104]:
#대략적인 데이터의 예측 차이를 출력
def summary_predcit_info(index, n_y, org_set, predict_y, n_x = None):
    print(org_set[index][0])
     
    #n_x 입력시 단어 개수 정도? 
    if n_x is not None:
        n_count = 0
        #소수점은 해당 단어가 호출되는 빈도 (전체 데이터셋 기준)
        for idx, n_data in enumerate(n_x[index]):
            if n_data:
                n_count += 1
        print(f"단어 수 {n_count}")
    
    print(f"실제 수정 파일 수 {len(org_set[index][1])}")
            
    # 인덱스와 키를 뒤집음: 인덱스 -> 경로
    reversed_path = {i: p for p, i in file_to_enum.items()}
    

    #예측 데이터
    n_count = 0
    for idx, n_data in enumerate(predict_y[index]):
        if 0.5 < n_data:
            n_count+=1
            print(f'{idx}:{n_data} -> ({reversed_path[idx]})' ) #예측 개수가 작으니 출력
    
    print(f"예측 라벨링 개수 {n_count}")
    
    return n_count

# is_label_predict_list = []

for idx in is_label_predict_list:
    print("=====",idx,"=======")
    summary_predcit_info(idx, n_x=test_x, n_y=test_y, predict_y=predict_y, org_set=test_msg_files)
    



===== 42 =======
Task #4111 [Surgical Guide] Bar/Window 사양 정의

-BarWindow 폴더 분리
-ToothRemove 수행시, Fixture 위치 이동시 Guide, Bar, Tag 데이터 모두 삭제
-UpdateSurface, 투명도 조절 분리(Guide, Bar, Tag)
-BarWindow 진입시 항상 Play 버튼
-Guide Creation, Bar and Window, ID Tag 진입시 선택된 LoopNum 가져옴
-Making Guide 수행 후 -> BarWindow, Tag 에 복사
단어 수 36
실제 수정 파일 수 18
18:0.6913949251174927 -> (D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APSurgicalGuide\ToolBarSurgicalGuide.cpp)
47:0.8694677948951721 -> (D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APSurgicalGuide\ActuatorSurgicalGuide.cpp)
207:0.5384683012962341 -> (D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APSurgicalGuide\CommandIDTag.h)
예측 라벨링 개수 3
===== 477 =======
Task #3957 (AP) 2-4 : ID Tag 임시적용

-1개 가이드에 1개 추가(Boolean X)
단어 수 10
실제 수정 파일 수 1
18:0.55435711145401 -> (D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APSurgicalGuide\ToolBarSurgicalGuide.cpp)
27:0.5271874666213989 -> (D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APSurgicalGuide\CommandIDTag.cpp)
47:0.873598933

In [101]:
def print_org_to_n(index):
    print(msg_files[index][0])
    print(f'->{x[index]} len {len(x[index])}')

    # 인덱스와 키를 뒤집음: 인덱스 -> 단어
    reversed_vocab = {i: word for word, i in vectorizer.vocabulary_.items()}
    
    #소수점은 해당 단어가 호출되는 빈도 (전체 데이터셋 기준)
    for idx, n_data in enumerate(x[index]):
        if n_data:
            print(f'{idx}:{n_data} -> ({reversed_vocab[idx]})' )
    
    print()
    for path in msg_files[index][1]:
        print(f'{path} : {file_to_enum[path]}')
    
    print(f'->{y[index]} len {len(y[index])}')
    
    # 인덱스와 키를 뒤집음: 인덱스 -> 경로
    reversed_path = {i: p for p, i in file_to_enum.items()}
    
    for idx, n_data in enumerate(y[index]):
        if n_data:
            print(f'{idx}:{n_data} -> ({reversed_path[idx]})' ) 
            
# for i in range(5):
#     print_org_to_n(50 * i)
#     print("========="*5)

print_org_to_n(test + 5)

NameError: name 'test' is not defined

In [41]:
# print(len(org_test_set))

org_train_set = msg_files[:-test_idx]
org_test_set = msg_files[-test_idx:]
# len(org_test_set)
print(org_test_set[0])
print(test_x[0])

('[CHANGED] Tooth Remove 버그 수정', ['D:\\dev\\AutoPlanning\\trunk\\AP_trunk_pure\\mod_APScanEdit\\ManipulatorToothRemove.cpp', 'D:\\dev\\AutoPlanning\\trunk\\AP_trunk_pure\\mod_APScanEdit\\ManipulatorToothRemove.h', 'D:\\dev\\AutoPlanning\\trunk\\AP_trunk_pure\\mod_APSurgicalGuide\\ManipulatorNamingBar.cpp'])
[0. 0. 0. ... 0. 0. 0.]


In [ ]:
from tensorflow.keras.models import Sequential

In [41]:
#4. 선형 학습
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Lambda, Dropout, BatchNormalization, LSTM

model = Sequential([
        Dense(4096, activation='relu', input_shape=(6538,)),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(138371, activation='sigmoid')
    ])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

size = len(n_msg_files)
test_rate = 0.2
test_idx = int(size * test_rate)

x = [pair[0] for pair in n_msg_files]
y = [pair[1] for pair in n_msg_files]

x = np.vstack(x)
y = np.vstack(y)

print(x.shape)  # (n, 6538)
print(y.shape)  # (n, 138371)

train_x = x[:-test_idx]
train_y = y[:-test_idx]

test_x = x[-test_idx:]
test_y = y[-test_idx:]

print(train_x.shape)  # (n, 6538)
print(train_y.shape)  # (n, 138371)


# 모델 학습
model.fit(train_x, train_y, epochs=10, batch_size=1)


# predict_y = self.scaler_y.inverse_transform(predict_yn.reshape(-1, 1)).flatten() 이미 정규화




(3888, 6538)
(3888, 138371)
(3111, 6538)
(3111, 138371)
Epoch 1/10
3111/3111 [==============================] - 81s 26ms/step - loss: 0.0105 - accuracy: 0.0119
Epoch 2/10
3111/3111 [==============================] - 80s 26ms/step - loss: 0.0058 - accuracy: 0.0193
Epoch 3/10
3111/3111 [==============================] - 81s 26ms/step - loss: 0.0044 - accuracy: 0.0257
Epoch 4/10
3111/3111 [==============================] - 80s 26ms/step - loss: 0.0031 - accuracy: 0.0347
Epoch 5/10
3111/3111 [==============================] - 82s 26ms/step - loss: 0.0037 - accuracy: 0.0466
Epoch 6/10
3111/3111 [==============================] - 82s 26ms/step - loss: 0.0031 - accuracy: 0.0408
Epoch 7/10
3111/3111 [==============================] - 81s 26ms/step - loss: 0.0031 - accuracy: 0.0453
Epoch 8/10
3111/3111 [==============================] - 80s 26ms/step - loss: 0.0020 - accuracy: 0.0559
Epoch 9/10
3111/3111 [==============================] - 80s 26ms/step - loss: 0.0027 - accuracy: 0.0588
Epoch 10

NameError: name 'self' is not defined

In [44]:
predict_yn = model.predict(test_x)

In [43]:
model.sa